# Matrix Factorization

In [4]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse 

class MF(object):
    """docstring for CF"""
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None, 
                 learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        self.lam = lam
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.print_every = print_every
        self.user_based = user_based
        # number of users and items. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1 
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        
        if Xinit is None: 
            self.X = np.random.randn(self.n_items, K)
        else:
            self.X = Xinit 
        
        if Winit is None: 
            self.W = np.random.randn(K, self.n_users)
        else: 
            self.W = Winit
            
        #self.all_users = self.Y_data[:,0] # all users (may be duplicated)
        self.n_ratings = Y_data.shape[0]
        # normalized data
        self.Y_data_n = self.Y_raw_data.copy()

    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
        else:
            user_col = 1
            item_col = 0 
            n_objects = self.n_items

        users = self.Y_raw_data[:, user_col] 
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col] 
            # and the corresponding ratings 
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings) 
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
            
    
    def loss(self):
        L = 0 
        for i in range(self.Y_data_n.shape[0]):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
            
        # regularization 
        L /= self.n_ratings
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L 

    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user n, and the corresponding ratings
        """
        # y = self.Y_data_n[:,0] # all users (may be duplicated)
        # item indices rated by user_id
        # need to +1 to user_id since in the rate_matrix, id starts from 1 
        # while index in python starts from 0
        ids = np.where(self.Y_data_n[:,0] == user_id)[0] 
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # index starts from 0 
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
        
        
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item m and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:,1] == item_id)[0] 
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
        
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + \
                                               self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
    
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + \
                        self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
    
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print ("iter =", it + 1, ", loss =", self.loss(), ", RMSE train =", rmse_train)
    
    
    def pred(self, u, i):
        """ 
        predict the rating of user u for item i 
        """
        u = int(u)
        i = int(i)
        
        if self.user_based:
            bias = self.mu[u]
        else: 
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias 
        if pred < 1:
            return 1 
        if pred > 5: 
            return 5 
        return pred 
        
    
    def pred_for_user(self, user_id):
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()              
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings= []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
        
        return predicted_ratings
    
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2 

        RMSE = np.sqrt(SE/n_tests)
        return RMSE


In [16]:
r_cols = ['userId', 'itemId', 'rating']

ratings_base = pd.read_csv('./ratings.csv', sep=',', names=r_cols, encoding='latin-1', header=0)

# Convert userID and itemID columns to integers
ratings_base['userId'] = ratings_base['userId'].astype(int)
ratings_base['itemId'] = ratings_base['itemId'].astype(int)

rate_train = ratings_base.to_numpy()

# Adjust indices so that they start from 0
rate_train[:, :2] -= 1

In [17]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit()
# evaluate on data
RMSE = rs.evaluate_RMSE(rate_train)
print ("\nUser-based MF, RMSE =", RMSE)

iter = 10 , loss = 12.01723774963348 , RMSE train = 1.238428830983639
iter = 20 , loss = 5.755035004252499 , RMSE train = 1.1157175391464231
iter = 30 , loss = 2.968231508275845 , RMSE train = 1.1087898577544388
iter = 40 , loss = 1.6938457222139358 , RMSE train = 1.1086614869417803
iter = 50 , loss = 1.1095494922882214 , RMSE train = 1.1086998619930535
iter = 60 , loss = 0.8415944302343801 , RMSE train = 1.1087119068581643
iter = 70 , loss = 0.7187112371310225 , RMSE train = 1.108714842708818
iter = 80 , loss = 0.6623579675429171 , RMSE train = 1.108715525725856
iter = 90 , loss = 0.6365149391218838 , RMSE train = 1.1087156823275275
iter = 100 , loss = 0.6246636221442141 , RMSE train = 1.1087157179525444

User-based MF, RMSE = 1.1087157179525444


In [18]:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on data
RMSE = rs.evaluate_RMSE(rate_train)
print ("\nItem-based MF, RMSE =", RMSE)

iter = 10 , loss = 11.847486656120967 , RMSE train = 1.0818093872949304
iter = 20 , loss = 5.5827333996123585 , RMSE train = 0.9456746613640713
iter = 30 , loss = 2.794623405908604 , RMSE train = 0.9382657614193841
iter = 40 , loss = 1.5197046872702187 , RMSE train = 0.9381022316421769
iter = 50 , loss = 0.9351804125034042 , RMSE train = 0.9381401820102356
iter = 60 , loss = 0.6671238477300292 , RMSE train = 0.9381531745307061
iter = 70 , loss = 0.5441946366499508 , RMSE train = 0.9381565143125821
iter = 80 , loss = 0.48782034253129775 , RMSE train = 0.9381573258516173
iter = 90 , loss = 0.46196767629214597 , RMSE train = 0.938157518939785
iter = 100 , loss = 0.45011193351115486 , RMSE train = 0.938157564303392

Item-based MF, RMSE = 0.938157564303392


In [19]:
rs = MF(rate_train, K = 2, lam = 0, print_every = 10, learning_rate = 1, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on  data
RMSE = rs.evaluate_RMSE(rate_train)
print ("\nItem-based MF, RMSE =", RMSE)

iter = 10 , loss = 1.3805388875000366 , RMSE train = 1.3409997146841588
iter = 20 , loss = 1.3455075409406052 , RMSE train = 1.3315408521056242
iter = 30 , loss = 1.3129409282425941 , RMSE train = 1.3224433938041646
iter = 40 , loss = 1.282568546645268 , RMSE train = 1.3137170837483485
iter = 50 , loss = 1.2541609812162464 , RMSE train = 1.305320584703306
iter = 60 , loss = 1.2275216708054344 , RMSE train = 1.2972299111891776
iter = 70 , loss = 1.2024807739702095 , RMSE train = 1.289435113025432
iter = 80 , loss = 1.1788904877480884 , RMSE train = 1.2819215654056402
iter = 90 , loss = 1.1566213995957406 , RMSE train = 1.2746717990374459
iter = 100 , loss = 1.1355595932324931 , RMSE train = 1.2677108465527178

Item-based MF, RMSE = 1.2677108465527178


In [22]:
RMSE = rs.evaluate_RMSE(rate_train)
RMSE

1.2677108465527178